In [ ]:
import os, argparse, json
import numpy as np
from pathlib import Path
from sim_motor import MotorParams, InvParams, SimConfig, FaultConfig, MotorInverterSim
from faults import default_scenarios


def gen_one(outdir, speed_rpm, load_torque, scen, idx, seed=0):
    np.random.seed(seed+idx)
    fs = 20000.0
    t_end = 2.0
    sc = SimConfig(fs=fs, t_end=t_end, speed_mech_rpm=speed_rpm, load_torque=load_torque)
    fc = FaultConfig(kind=scen.kind, phase=scen.phase, severity=scen.severity)
    sim = MotorInverterSim(MotorParams(), InvParams(), sc, fc)
    data = sim.run()
    # add sensor noise
    for k in ['ia','ib','ic','va','vb','vc']:
        data[k] = data[k] + 0.01*np.random.randn(len(data[k]))
    meta = {
        'label': scen.kind, 'phase': scen.phase, 'severity': scen.severity,
        'speed_rpm': speed_rpm, 'load_torque': load_torque, 'fs': data['fs']
    }
    np.savez(os.path.join(outdir, f"sample_{idx:05d}.npz"), **data)
    with open(os.path.join(outdir, f"sample_{idx:05d}.json"), 'w') as f:
        json.dump(meta, f)


def main():
    ap = argparse.ArgumentParser()
    ap.add_argument('--out', type=str, default='data/raw')
    ap.add_argument('--n_scen', type=int, default=60)
    ap.add_argument('--seed', type=int, default=0)
    args = ap.parse_args()

    Path(args.out).mkdir(parents=True, exist_ok=True)

    speeds = np.linspace(800, 2600, 6)  # create domain variety
    loads  = np.linspace(6, 14, 5)
    scens = default_scenarios()

    idx = 0
    for i in range(args.n_scen):
        speed = float(np.random.choice(speeds))
        load  = float(np.random.choice(loads))
        scen  = np.random.choice(scens)
        gen_one(args.out, speed, load, scen, idx, seed=args.seed)
        idx += 1

if _name_ == '_main_':
    main()